In [13]:
#imports
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

Read in the cleaned data from the previous notebook. I will split into train and validate (stratifying according to the target label) and transform using the tf-idf vectorizer.

In [2]:
#read in df
df = pd.read_csv('clean_stemmed.csv', index_col=0)
df.head()

,label,text
0,Thyroid_Cancer,thyroid surgeri children singl institut osama ...
1,Thyroid_Cancer,adopt strategi use prior year base four exclus...
2,Thyroid_Cancer,coronari arterybypass graft thrombosi brin bri...
3,Thyroid_Cancer,solitari plasmacytoma sp skull uncommon clinic...
4,Thyroid_Cancer,studi aim investig serum matrix metalloprotein...


In [3]:
#labels are balanced enough
df.label.value_counts()

Lung_Cancer       452
Thyroid_Cancer    283
Colon_Cancer      261
Name: label, dtype: int64

In [4]:
#split
train, val = train_test_split(df, train_size=0.7,
                              stratify=df.label)

train.shape, val.shape

((697, 2), (299, 2))

In [5]:
#transform train using vectorizer
vectorizer = TfidfVectorizer()
train_vec = vectorizer.fit_transform(train.text)

train_vec.shape

(697, 126262)

There are about 125k unique words found in my body of text. I'm going to mess with some of the parameters of the vectorizer to bring that value down. I'll only select the top 1,000 most frequently occuring words.

In [6]:
#round 2
vectorizer = TfidfVectorizer(max_features=1_000)
train_vec = vectorizer.fit_transform(train.text)

train_vec.shape

(697, 1000)

In [7]:
train_vec = pd.DataFrame(train_vec.toarray(), columns=vectorizer.vocabulary_.keys())
train_vec.head()

,novel,coronaviru,sarscov,found,caus,highli,diseas,character,covid,global,...,circrna,ncih,learn,mcf,mdamb,irradi,mtbhsp,scfvmtbhsp,keloid,pax
0,0.0,0.0,0.0,0.0,0.016183,0.0,0.008092,0.008113,0.0,0.016448,...,0.0,0.0,0.010980,0.015566,0.051056,0.000000,0.0,0.000000,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.013025,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.017674,0.012528,0.010273,0.053948,0.0,0.000000,0.007970,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.011251,0.000000,0.000000,0.000000,0.0,0.017905,0.020294,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0


Still a lot of features. I'll use recursive feature elimination with a basic decision tree to reduce this to 50.

In [8]:
#run rfe
tree = DecisionTreeClassifier(random_state=42)
selector = RFE(tree, n_features_to_select=50, step=50)
selector.fit(train_vec, train.label)

RFE(estimator=DecisionTreeClassifier(random_state=42), n_features_to_select=50,
    step=50)

Here we can see the final 50 features that have been selected. Each row is a document from the train set. I will use these features to predict the label.

In [9]:
final_train = train_vec.loc[:, selector.support_]
final_train.head()

,covid,imag,case,identif,isol,avail,literatur,current,laboratori,limit,...,aal,deplet,infiltr,environment,oxid,accumul,wwwfrontiersinaugust,sirna,identi,learn
0,0.0,0.000000,0.012566,0.015720,0.000000,0.000000,0.0,0.008092,0.000000,0.007362,...,0.0,0.0,0.026098,0.012435,0.004561,0.009136,0.011984,0.048444,0.0,0.010980
1,0.0,0.031172,0.010113,0.037955,0.012465,0.055892,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.046706,0.022026,0.051473,0.000000,0.015596,0.0,0.017674
2,0.0,0.029766,0.000000,0.000000,0.000000,0.000000,0.0,0.016583,0.020804,0.000000,...,0.0,0.0,0.017828,0.016990,0.177602,0.056172,0.000000,0.000000,0.0,0.011251
3,0.0,0.000000,0.000000,0.000000,0.000000,0.026236,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,0.0,0.014357,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.012899,0.000000,0.040579,0.000000,0.000000,0.000000,0.0,0.000000


In [16]:
le = LabelEncoder()
y_train = le.fit_transform(train.label)

First, let's establish a baseline accuracy. The baseline will be calculated by predicting the most prevalent class every time.

In [10]:
train.label.value_counts()

Lung_Cancer       316
Thyroid_Cancer    198
Colon_Cancer      183
Name: label, dtype: int64

In [11]:
round(train.label.value_counts().max() / train.label.value_counts().sum() * 100, 2)

45.34

Our baseline accuracy is about 45% when guessing the most prevalent class (Lung_Cancer) for every document. Let's see if we can beat that baseline with an xgboost model with all default parameters.

In [23]:
clf = XGBClassifier(random_state=42)
clf.fit(final_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [24]:
clf.score(final_train, y_train)

1.0

Perfect accuracy on the train dataset.. I'm concerned. Let's prepare the validate dataset the same way as train, and see how our model fares on unseen data.

In [25]:
val_vec = vectorizer.transform(val.text)
val_vec = pd.DataFrame(val_vec.toarray(), columns=vectorizer.vocabulary_.keys())
final_val = val_vec.loc[:, selector.support_]
final_val.head()

,covid,imag,case,identif,isol,avail,literatur,current,laboratori,limit,...,aal,deplet,infiltr,environment,oxid,accumul,wwwfrontiersinaugust,sirna,identi,learn
0,0.0,0.037005,0.00000,0.004291,0.016911,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.445275,0.018997,0.000000,0.034862,0.024940,0.00000,0.0,0.000000,0.005995
1,0.0,0.018341,0.00000,0.014888,0.000000,0.078925,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.062812,0.000000,0.000000,0.00000,0.0,0.000000,0.000000
2,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.098527,0.03231,0.0,0.000000,0.088807
3,0.0,0.007259,0.00471,0.014731,0.000000,0.000000,0.0,0.006066,0.0,0.000000,...,0.0,0.005879,0.000000,0.040397,0.000000,0.054793,0.00000,0.0,0.046247,0.004116
4,0.0,0.011694,0.00000,0.000000,0.000000,0.000000,0.0,0.009772,0.0,0.044454,...,0.0,0.009471,0.015759,0.150186,0.000000,0.000000,0.00000,0.0,0.000000,0.000000


In [26]:
y_val = le.transform(val.label)

Now that our validate dataset has been prepared in the same manner as train, it can be used to generate predictions from our trained model.

In [27]:
clf.score(final_val, y_val)

0.8862876254180602

Over 88% accuracy! This is a great start for the project. We have almost doubled the baseline accuracy (45%). This proves the project has merit, and future feature selection / model fine-tuning could improve the result. Now, the goal is to pickle this model and make it callable from the command line.